# Token Insider Analysis Notebook

This notebook demonstrates how to use SolanaGuard to analyze tokens for insider trading patterns, team wallet identification, and token creator behavior.

## Overview

Token insider analysis focuses on identifying patterns that may indicate:  
1. **Team wallets**: Addresses controlled by project team members  
2. **Insider trading**: Trading based on non-public information  
3. **Rug pull preparation**: Token creators preparing to abandon the project  
4. **Wash trading**: Creating fake trading volume  

In [ ]:
# Import required libraries
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime, timedelta

# Add project root to path
sys.path.append('../..')

# Import SolanaGuard modules
from data_collection.collectors.helius_collector import HeliusCollector
from data_collection.collectors.rugcheck_collector import RugCheckCollector
from data_collection.collectors.vybe_collector import VybeCollector
from utils.graph_utils import build_token_insider_graph
from utils.risk_scoring import calculate_token_risk
from utils.visualization import visualize_token_activity, visualize_risk_score

# Configure plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

ModuleNotFoundError: No module named 'collectors'

## Initialize API Collectors

First, we initialize the necessary API collectors to gather token data:

In [ ]:
# Initialize collectors
rugcheck = RugCheckCollector()
vybe = VybeCollector()
helius = HeliusCollector()

print("Collectors initialized successfully")

## Identify Suspicious Tokens

Let's start by identifying potentially suspicious tokens from trending data:

In [ ]:
# If the API doesn't work, we can specify known tokens of interest directly
predefined_suspicious_tokens = [
    {
        "mint": "7dHbWXmci3dT8UFYWYZweBLXgycu7Y3iL6trKn1Y7ARj", # Bonk
        "name": "Bonk",
        "symbol": "BONK",
        "risk_score": 70,
        "risks": ["High concentration", "Team wallet selling"]
    },
    {
        "mint": "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v", # USDC on Solana
        "name": "USD Coin",
        "symbol": "USDC",
        "risk_score": 30,
        "risks": ["Centralized control"]
    },
    {
        "mint": "4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R", # Raydium
        "name": "Raydium",
        "symbol": "RAY",
        "risk_score": 45,
        "risks": ["Concentration risk"]
    }
]

# Get trending tokens data
try:
    trending_tokens = rugcheck.get_trending_tokens()
    print(f"Retrieved {len(trending_tokens)} trending tokens")
except Exception as e:
    print(f"Error getting trending tokens: {e}")
    trending_tokens = []
    
# If API call fails, use our predefined tokens
if not trending_tokens:
    print("Using predefined token list as fallback")
    suspicious_tokens = predefined_suspicious_tokens
else:
    # Function to filter suspicious tokens
    def filter_suspicious_tokens(tokens, max_tokens=10):
        suspicious_tokens = []
        
        for token in tokens[:20]:  # Check top 20 tokens
            token_mint = token.get("mint")
            if not token_mint:
                continue
                
            # Get token summary report
            try:
                summary = rugcheck.get_token_report_summary(token_mint, cache_only=True)
                
                # Check if token has high risk score
                if summary.get("score_normalised", 0) > 60:
                    suspicious_tokens.append({
                        "mint": token_mint,
                        "name": token.get("name", "Unknown"),
                        "symbol": token.get("symbol", "???"),
                        "risk_score": summary.get("score_normalised", 0),
                        "risks": summary.get("risks", [])
                    })
                    
                    if len(suspicious_tokens) >= max_tokens:
                        break
            except Exception as e:
                print(f"Error analyzing token {token_mint}: {e}")
        
        return suspicious_tokens

    suspicious_tokens = filter_suspicious_tokens(trending_tokens)

print(f"\nIdentified {len(suspicious_tokens)} suspicious tokens:")
for token in suspicious_tokens:
    print(f"- {token['symbol']} ({token['mint'][:10]}...): Risk score {token['risk_score']}")

## Analyze Token Creator Patterns

Let's analyze the creator wallet patterns for these suspicious tokens:

In [ ]:
# Function to analyze token creator patterns
def analyze_token_creator(token_mint):
    print(f"\nAnalyzing creator for token: {token_mint}")
    
    # Get full token report
    try:
        token_report = rugcheck.get_token_report(token_mint)
        creator_address = token_report.get("creator")
        if not creator_address:
            print("Creator address not found")
            return {}
            
        print(f"Creator address: {creator_address}")
        
        # Get creator's other tokens
        creator_tokens = token_report.get("creatorTokens", [])
        print(f"Creator has launched {len(creator_tokens)} tokens")
        
        # Get token's metadata
        token_meta = token_report.get("tokenMeta", {})
        token_name = token_meta.get("name", "Unknown")
        token_symbol = token_meta.get("symbol", "???")
        
        # Get transaction history for creator
        tx_history = helius.fetch_transaction_history(creator_address, limit=100)
        print(f"Fetched {len(tx_history)} transactions for creator")
        
        # Get token transfers for creator
        token_transfers = helius.analyze_token_transfers(creator_address, limit=100)
        print(f"Fetched {len(token_transfers)} token transfers for creator")
        
        # Analyze other tokens from same creator
        other_tokens_risk = []
        for creator_token in creator_tokens:
            other_mint = creator_token.get("mint")
            if other_mint and other_mint != token_mint:
                try:
                    other_summary = rugcheck.get_token_report_summary(other_mint, cache_only=True)
                    other_tokens_risk.append({
                        "mint": other_mint,
                        "risk_score": other_summary.get("score_normalised", 0),
                        "created_at": creator_token.get("createdAt"),
                        "market_cap": creator_token.get("marketCap", 0)
                    })
                except Exception as e:
                    print(f"Error analyzing other token {other_mint}: {e}")
        
        # Calculate average risk score for creator's tokens
        avg_risk = np.mean([t["risk_score"] for t in other_tokens_risk]) if other_tokens_risk else 0
        print(f"Average risk score for creator's other tokens: {avg_risk:.2f}")
        
        # Determine if creator has history of rug pulls
        high_risk_tokens = [t for t in other_tokens_risk if t["risk_score"] > 80]
        has_rug_history = len(high_risk_tokens) > 0
        if has_rug_history:
            print(f"Creator has {len(high_risk_tokens)} high-risk tokens (potential rug pulls)")
        
        return {
            "token_mint": token_mint,
            "token_name": token_name,
            "token_symbol": token_symbol,
            "creator_address": creator_address,
            "creator_tokens_count": len(creator_tokens),
            "other_tokens_risk": other_tokens_risk,
            "avg_risk_score": avg_risk,
            "has_rug_history": has_rug_history,
            "tx_history": tx_history,
            "token_transfers": token_transfers
        }
    except Exception as e:
        print(f"Error getting token report: {e}")
        return {}

# Analyze creators for suspicious tokens
creator_analyses = {}
for token in suspicious_tokens:
    creator_analyses[token["mint"]] = analyze_token_creator(token["mint"])

## Insider Network Analysis

Now, let's analyze the insider networks for these tokens:

In [ ]:
# Function to analyze token insider network
def analyze_token_insiders(token_mint):
    print(f"\nAnalyzing insider network for token: {token_mint}")
    
    # Get insider graph
    try:
        insider_graph_data = rugcheck.get_token_insider_graph(token_mint)
        
        if not insider_graph_data:
            print("No insider graph data available")
            return {}
        
        # Build insider graph
        graph = build_token_insider_graph(insider_graph_data)
        
        # Get insider metrics
        node_count = len(graph.graph.nodes())
        edge_count = len(graph.graph.edges())
        print(f"Insider graph has {node_count} nodes and {edge_count} edges")
        
        # Get top holders
        try:
            token_report = rugcheck.get_token_report(token_mint)
            top_holders = token_report.get("topHolders", [])
            print(f"Retrieved {len(top_holders)} top holders")
        except Exception as e:
            print(f"Error getting top holders: {e}")
            top_holders = []
        
        # Calculate holder concentration
        top_holder_pct = top_holders[0].get("pct", 0) if top_holders else 0
        top5_pct = sum(h.get("pct", 0) for h in top_holders[:5]) if len(top_holders) >= 5 else sum(h.get("pct", 0) for h in top_holders)
        
        print(f"Top holder controls {top_holder_pct:.2f}% of supply")
        print(f"Top 5 holders control {top5_pct:.2f}% of supply")
        
        # Get insider marked holders
        insider_holders = [h for h in top_holders if h.get("insider", False)]
        insider_pct = sum(h.get("pct", 0) for h in insider_holders)
        
        print(f"Identified {len(insider_holders)} insider wallets controlling {insider_pct:.2f}% of supply")
        
        # Analyze graph communities
        communities = graph.identify_communities()
        print(f"Detected {len(communities)} communities in the insider network")
        
        # Analyze token liquidity
        markets = token_report.get("markets", [])
        total_liquidity = 0
        liquidity_locked_pct = 0
        
        for market in markets:
            lp_data = market.get("lp", {})
            if lp_data:
                total_liquidity += lp_data.get("baseUSD", 0) + lp_data.get("quoteUSD", 0)
                liquidity_locked_pct = max(liquidity_locked_pct, lp_data.get("lpLockedPct", 0))
        
        print(f"Total liquidity: ${total_liquidity:.2f}")
        print(f"Liquidity locked: {liquidity_locked_pct:.2f}%")
        
        # Identify suspicious patterns
        suspicious_patterns = graph.detect_suspicious_patterns()
        print(f"Detected {len(suspicious_patterns)} suspicious patterns in the insider network")
        
        for pattern in suspicious_patterns:
            print(f"- {pattern['type']}: {pattern['description']} (risk: {pattern['risk_score']})")
        
        # Save graph for visualization
        graph_json = graph.export_to_json()
        with open(f"../../data/output/insider_graph_{token_mint}.json", "w") as f:
            json.dump(graph_json, f)
        
        # Return insider analysis results
        return {
            "token_mint": token_mint,
            "node_count": node_count,
            "edge_count": edge_count,
            "top_holders": top_holders,
            "top_holder_pct": top_holder_pct,
            "top5_pct": top5_pct,
            "insider_holders": insider_holders,
            "insider_pct": insider_pct,
            "communities": communities,
            "total_liquidity": total_liquidity,
            "liquidity_locked_pct": liquidity_locked_pct,
            "suspicious_patterns": suspicious_patterns,
            "graph": graph
        }
    except Exception as e:
        print(f"Error analyzing insider network: {e}")
        return {}

# Analyze insider networks for suspicious tokens
insider_analyses = {}
for token in suspicious_tokens:
    insider_analyses[token["mint"]] = analyze_token_insiders(token["mint"])

## Token Activity Analysis

Let's analyze token activity patterns to identify potential wash trading or price manipulation:

In [ ]:
# Function to analyze token activity
def analyze_token_activity(token_mint):
    print(f"\nAnalyzing token activity for: {token_mint}")
    
    # Get token details
    try:
        token_details = vybe.get_token_details(token_mint)
        print(f"Retrieved token details for {token_details.get('name', 'Unknown')} ({token_details.get('symbol', '???')})")
    except Exception as e:
        print(f"Error getting token details: {e}")
        token_details = {}
    
    # Get token price history
    try:
        price_data = vybe.get_token_price_ohlcv(
            token_mint, 
            resolution="1d", 
            time_start=int((datetime.now() - timedelta(days=30)).timestamp()),
            time_end=int(datetime.now().timestamp())
        )
        price_history = price_data.get("data", [])
        print(f"Retrieved {len(price_history)} price data points")
    except Exception as e:
        print(f"Error getting price history: {e}")
        price_history = []
    
    # Get token holders
    try:
        holders_data = vybe.get_token_top_holders(token_mint)
        holders = holders_data.get("data", [])
        print(f"Retrieved {len(holders)} token holders")
    except Exception as e:
        print(f"Error getting token holders: {e}")
        holders = []
    
    # Get token transfers
    try:
        transfers_data = vybe.get_token_transfers(
            mint_address=token_mint,
            time_start=int((datetime.now() - timedelta(days=7)).timestamp()),
            time_end=int(datetime.now().timestamp()),
            limit=500
        )
        transfers = transfers_data.get("data", [])
        print(f"Retrieved {len(transfers)} token transfers")
    except Exception as e:
        print(f"Error getting token transfers: {e}")
        transfers = []
    
    # Analyze transfer patterns for wash trading
    wash_trading_indicators = 0
    wash_trading_reasons = []
    
    if transfers and len(transfers) > 10:
        # Check for circular transfers
        transfer_pairs = set()
        for tx in transfers:
            sender = tx.get("sender_address")
            receiver = tx.get("receiver_address")
            
            if sender and receiver:
                if (receiver, sender) in transfer_pairs:
                    wash_trading_indicators += 1
                    wash_trading_reasons.append(f"Circular transfer between {sender[:6]}... and {receiver[:6]}...")
                
                transfer_pairs.add((sender, receiver))
        
        # Check for unusual timing patterns
        if len(transfers) >= 3:
            # Convert timestamps to datetime objects
            timestamps = sorted([tx.get("block_time", 0) for tx in transfers if tx.get("block_time")])
            
            if len(timestamps) >= 3:
                # Calculate time differences
                time_diffs = [timestamps[i+1] - timestamps[i] for i in range(len(timestamps)-1)]
                
                # Check for extremely regular time intervals
                if len(time_diffs) > 5:
                    avg_diff = np.mean(time_diffs)
                    std_diff = np.std(time_diffs)
                    
                    if std_diff / avg_diff < 0.1 and len(time_diffs) > 10:  # Very consistent timing
                        wash_trading_indicators += 2
                        wash_trading_reasons.append(f"Extremely regular transaction timing (CV: {std_diff/avg_diff:.4f})")
    
    # Check for unusual volume patterns in price data
    if price_history and len(price_history) > 5:
        volumes = [p.get("volume", 0) for p in price_history]
        
        # Check for sudden volume spikes
        for i in range(1, len(volumes)):
            if volumes[i] > 5 * volumes[i-1] and volumes[i] > 0:
                wash_trading_indicators += 1
                wash_trading_reasons.append(f"Sudden volume spike ({volumes[i]/volumes[i-1]:.1f}x increase)")
    
    print(f"Wash trading indicators: {wash_trading_indicators}")
    if wash_trading_reasons:
        for reason in wash_trading_reasons[:3]:  # Show top 3 reasons
            print(f"- {reason}")
    
    # Visualize token activity
    token_data = {
        "token_mint": token_mint,
        "token_name": token_details.get("name", "Unknown"),
        "token_symbol": token_details.get("symbol", "???"),
        "price_history": price_history,
        "holder_data": holders,
        "volume_data": price_history  # Use the same data for volume
    }
    
    # Add risk data if available
    for token_info in suspicious_tokens:
        if token_info["mint"] == token_mint:
            token_data["risk_score"] = token_info["risk_score"]
            token_data["risk_level"] = "high" if token_info["risk_score"] > 75 else "medium"
            break
    
    # Generate visualization
    viz_file = visualize_token_activity(token_data, output_file=f"../../data/visualizations/token_activity_{token_mint}.png")
    
    return {
        "token_mint": token_mint,
        "token_details": token_details,
        "price_history": price_history,
        "holders": holders,
        "transfers": transfers,
        "wash_trading_indicators": wash_trading_indicators,
        "wash_trading_reasons": wash_trading_reasons,
        "visualization_file": viz_file
    }

# Analyze token activity for suspicious tokens
activity_analyses = {}
for token in suspicious_tokens:
    activity_analyses[token["mint"]] = analyze_token_activity(token["mint"])

## Token Risk Assessment

Let's calculate comprehensive risk scores for each token:

In [ ]:
# Calculate comprehensive risk scores for each token
token_risk_scores = {}

for token in suspicious_tokens:
    token_mint = token["mint"]
    
    # Gather all data needed for risk calculation
    token_details = activity_analyses.get(token_mint, {}).get("token_details", {})
    token_holders = activity_analyses.get(token_mint, {}).get("holders", [])
    
    # Convert transfers to DataFrame if available
    transfers = activity_analyses.get(token_mint, {}).get("transfers", [])
    token_transfers = pd.DataFrame(transfers) if transfers else pd.DataFrame()
    
    # Get insider data
    token_insider_data = insider_analyses.get(token_mint, {})
    
    # Get RugCheck risk data
    rugcheck_risk = pd.DataFrame([token]) if "risks" in token else pd.DataFrame()
    
    # Calculate risk score
    risk_score = calculate_token_risk(
        token_mint,
        token_details,
        token_holders,
        token_transfers,
        token_insider_data,
        rugcheck_risk
    )
    
    token_risk_scores[token_mint] = risk_score
    
    print(f"\nRisk assessment for {token['symbol']} ({token_mint[:10]}...):")
    print(f"- Overall score: {risk_score['risk_score']:.2f}")
    print(f"- Risk level: {risk_score['risk_level']}")
    print("- Top risk factors:")
    for factor in risk_score['risk_factors'][:3]:
        print(f"  * {factor['name']}: {factor['description']} (score: {factor['score']})")
    
    # Generate risk visualization
    visualize_risk_score(risk_score, output_file=f"../../data/visualizations/risk_score_{token_mint}.png")

## Rug Pull Likelihood Assessment

Let's assess the likelihood of a rug pull for each token:

In [ ]:
# Function to assess rug pull likelihood
def assess_rug_pull_likelihood(token_mint):
    # Get relevant analyses
    creator_data = creator_analyses.get(token_mint, {})
    insider_data = insider_analyses.get(token_mint, {})
    activity_data = activity_analyses.get(token_mint, {})
    risk_score = token_risk_scores.get(token_mint, {})
    
    # Initialize scores
    team_credibility_score = 0
    liquidity_risk_score = 0
    token_structure_score = 0
    trading_pattern_score = 0
    
    # Calculate team credibility score (0-100, higher is worse)
    if creator_data:
        # Check if creator has history of rug pulls
        if creator_data.get("has_rug_history"):
            team_credibility_score += 80
        
        # Check creator's average token risk score
        avg_risk = creator_data.get("avg_risk_score", 0)
        team_credibility_score += min(40, avg_risk * 0.5)
        
        # Multiple tokens is slightly suspicious
        token_count = creator_data.get("creator_tokens_count", 0)
        if token_count > 10:
            team_credibility_score += 30
        elif token_count > 5:
            team_credibility_score += 20
        elif token_count > 2:
            team_credibility_score += 10
    
    # Calculate liquidity risk score
    if insider_data:
        # Check liquidity locked percentage
        locked_pct = insider_data.get("liquidity_locked_pct", 0)
        if locked_pct < 50:
            liquidity_risk_score += 80  # Very high risk if <50% locked
        elif locked_pct < 80:
            liquidity_risk_score += 40  # Medium risk if <80% locked
        
        # Check total liquidity
        total_liquidity = insider_data.get("total_liquidity", 0)
        if total_liquidity < 1000:
            liquidity_risk_score += 80  # Very high risk if <$1k
        elif total_liquidity < 10000:
            liquidity_risk_score += 40  # Medium risk if <$10k
    
    # Calculate token structure score
    token_report = None
    try:
        token_report = rugcheck.get_token_report(token_mint)
    except:
        pass
    
    if token_report:
        # Check mint authority
        if token_report.get("mintAuthority"):
            token_structure_score += 50  # High risk if mint authority exists
        
        # Check freeze authority
        if token_report.get("freezeAuthority"):
            token_structure_score += 30  # Medium risk if freeze authority exists
        
        # Check token program
        token_program = token_report.get("tokenProgram")
        if token_program != "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA":  # Not standard SPL
            token_structure_score += 30  # Non-standard token program is risky
    
    # Calculate trading pattern score
    if activity_data:
        # Check wash trading indicators
        wash_indicators = activity_data.get("wash_trading_indicators", 0)
        if wash_indicators > 5:
            trading_pattern_score += 80  # Very high risk if many wash indicators
        elif wash_indicators > 2:
            trading_pattern_score += 40  # Medium risk if some wash indicators
    
    # Calculate overall rug pull likelihood
    # Weighted average of component scores
    overall_score = (
        team_credibility_score * 0.3 +
        liquidity_risk_score * 0.4 +
        token_structure_score * 0.2 +
        trading_pattern_score * 0.1
    )
    
    # Determine likelihood category
    if overall_score >= 80:
        likelihood = "Very High"
    elif overall_score >= 60:
        likelihood = "High"
    elif overall_score >= 40:
        likelihood = "Medium"
    elif overall_score >= 20:
        likelihood = "Low"
    else:
        likelihood = "Very Low"
    
    return {
        "token_mint": token_mint,
        "rug_pull_likelihood": likelihood,
        "overall_score": overall_score,
        "team_credibility_score": team_credibility_score,
        "liquidity_risk_score": liquidity_risk_score,
        "token_structure_score": token_structure_score,
        "trading_pattern_score": trading_pattern_score,
        "reasons": {
            "team_credibility": [
                "Creator has history of rug pulls" if creator_data.get("has_rug_history") else None,
                f"Creator has launched multiple tokens ({creator_data.get('creator_tokens_count', 0)})" if creator_data.get("creator_tokens_count", 0) > 1 else None,
                f"Creator's tokens have high average risk ({creator_data.get('avg_risk_score', 0):.2f})" if creator_data.get("avg_risk_score", 0) > 50 else None
            ],
            "liquidity_risk": [
                f"Low liquidity lock percentage ({insider_data.get('liquidity_locked_pct', 0):.2f}%)" if insider_data.get("liquidity_locked_pct", 100) < 80 else None,
                f"Low total liquidity (${insider_data.get('total_liquidity', 0):.2f})" if insider_data.get("total_liquidity", 100000) < 10000 else None
            ],
            "token_structure": [
                "Mint authority is active" if token_report and token_report.get("mintAuthority") else None,
                "Freeze authority is active" if token_report and token_report.get("freezeAuthority") else None,
                "Non-standard token program" if token_report and token_report.get("tokenProgram") != "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA" else None
            ],
            "trading_pattern": [
                f"Wash trading indicators detected ({activity_data.get('wash_trading_indicators', 0)})" if activity_data.get("wash_trading_indicators", 0) > 0 else None
            ]
        }
    }

# Assess rug pull likelihood for each token
rug_pull_assessments = {}
for token in suspicious_tokens:
    token_mint = token["mint"]
    assessment = assess_rug_pull_likelihood(token_mint)
    rug_pull_assessments[token_mint] = assessment
    
    print(f"\nRug pull assessment for {token['symbol']} ({token_mint[:10]}...):")
    print(f"- Likelihood: {assessment['rug_pull_likelihood']} ({assessment['overall_score']:.2f}/100)")
    print("- Key factors:")
    
    for category, reasons in assessment["reasons"].items():
        valid_reasons = [r for r in reasons if r]
        if valid_reasons:
            print(f"  * {category.replace('_', ' ').title()}:")
            for reason in valid_reasons:
                print(f"    - {reason}")

## Conclusions and Report Generation

Let's summarize our findings and generate a report:

In [ ]:
# Generate summary report
report = ["# Token Insider Analysis Report\n"]
report.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

report.append("## Tokens Analyzed\n")
for token in suspicious_tokens:
    token_mint = token["mint"]
    risk_level = token_risk_scores[token_mint]['risk_level'] if token_mint in token_risk_scores else "Unknown"
    rug_likelihood = rug_pull_assessments[token_mint]['rug_pull_likelihood'] if token_mint in rug_pull_assessments else "Unknown"
    report.append(f"- **{token['symbol']}** ({token_mint[:10]}...): Risk Level: {risk_level}, Rug Pull Likelihood: {rug_likelihood}")

for token in suspicious_tokens:
    token_mint = token["mint"]
    token_symbol = token["symbol"]
    token_name = token.get("name", "Unknown")
    
    report.append(f"\n## Analysis for {token_symbol} ({token_name})\n")
    report.append(f"Mint Address: `{token_mint}`\n")
    
    # Risk assessment
    if token_mint in token_risk_scores:
        report.append(f"### Risk Assessment\n")
        report.append(f"- Overall risk score: {token_risk_scores[token_mint]['risk_score']:.2f}")
        report.append(f"- Risk level: {token_risk_scores[token_mint]['risk_level']}\n")
        
        report.append("#### Risk Factors\n")
        for factor in token_risk_scores[token_mint]['risk_factors']:
            report.append(f"- **{factor['name']}**: {factor['description']} (score: {factor['score']})")
        
        report.append(f"\n![Risk Score](../../data/visualizations/risk_score_{token_mint}.png)")
    
    # Creator analysis
    if token_mint in creator_analyses and creator_analyses[token_mint]:
        creator_data = creator_analyses[token_mint]
        creator_address = creator_data.get("creator_address")
        
        if creator_address:
            report.append(f"\n### Creator Analysis\n")
            report.append(f"- Creator address: `{creator_address}`")
            report.append(f"- Total tokens created: {creator_data.get('creator_tokens_count', 0)}")
            report.append(f"- Average risk of other tokens: {creator_data.get('avg_risk_score', 0):.2f}")
            report.append(f"- History of rug pulls: {'Yes' if creator_data.get('has_rug_history') else 'No'}\n")
            
            other_tokens = creator_data.get("other_tokens_risk", [])
            if other_tokens:
                report.append("#### Other Tokens from Same Creator\n")
                for other_token in other_tokens[:5]:  # Show top 5
                    report.append(f"- `{other_token['mint'][:10]}...`: Risk score {other_token['risk_score']:.2f}")
    
    # Insider analysis
    if token_mint in insider_analyses and insider_analyses[token_mint]:
        insider_data = insider_analyses[token_mint]
        
        report.append(f"\n### Insider Network Analysis\n")
        report.append(f"- Network size: {insider_data.get('node_count', 0)} addresses, {insider_data.get('edge_count', 0)} connections")
        report.append(f"- Communities detected: {len(insider_data.get('communities', {}))}")
        report.append(f"- Top holder concentration: {insider_data.get('top_holder_pct', 0):.2f}%")
        report.append(f"- Top 5 holders concentration: {insider_data.get('top5_pct', 0):.2f}%")
        report.append(f"- Insider wallets control: {insider_data.get('insider_pct', 0):.2f}% of supply")
        report.append(f"- Total liquidity: ${insider_data.get('total_liquidity', 0):.2f}")
        report.append(f"- Liquidity locked: {insider_data.get('liquidity_locked_pct', 0):.2f}%\n")
        
        suspicious_patterns = insider_data.get("suspicious_patterns", [])
        if suspicious_patterns:
            report.append("#### Suspicious Patterns Detected\n")
            for pattern in suspicious_patterns:
                report.append(f"- **{pattern['type']}**: {pattern['description']} (risk: {pattern['risk_score']})")
    
    # Token activity analysis
    if token_mint in activity_analyses and activity_analyses[token_mint]:
        activity_data = activity_analyses[token_mint]
        
        report.append(f"\n### Token Activity Analysis\n")
        report.append(f"- Wash trading indicators: {activity_data.get('wash_trading_indicators', 0)}\n")
        
        wash_reasons = activity_data.get("wash_trading_reasons", [])
        if wash_reasons:
            report.append("#### Potential Wash Trading Evidence\n")
            for reason in wash_reasons:
                report.append(f"- {reason}")
        
        report.append(f"\n![Token Activity](../../data/visualizations/token_activity_{token_mint}.png)")
    
    # Rug pull assessment
    if token_mint in rug_pull_assessments:
        assessment = rug_pull_assessments[token_mint]
        
        report.append(f"\n### Rug Pull Assessment\n")
        report.append(f"- **Likelihood**: {assessment['rug_pull_likelihood']} ({assessment['overall_score']:.2f}/100)")
        report.append(f"- Team credibility score: {assessment['team_credibility_score']:.2f}/100")
        report.append(f"- Liquidity risk score: {assessment['liquidity_risk_score']:.2f}/100")
        report.append(f"- Token structure score: {assessment['token_structure_score']:.2f}/100")
        report.append(f"- Trading pattern score: {assessment['trading_pattern_score']:.2f}/100\n")
        
        report.append("#### Key Risk Factors\n")
        for category, reasons in assessment["reasons"].items():
            valid_reasons = [r for r in reasons if r]
            if valid_reasons:
                report.append(f"**{category.replace('_', ' ').title()}**:")
                for reason in valid_reasons:
                    report.append(f"- {reason}")
                report.append("")

# Save report to file
report_path = f"../../reports/token_insider_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
os.makedirs("../../reports", exist_ok=True)

with open(report_path, "w") as f:
    f.write("\n".join(report))

print(f"\nReport generated and saved to {report_path}")

# Display report in notebook
from IPython.display import Markdown
display(Markdown("\n".join(report)))